In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image


In [3]:
# Define data generators
batch_size = 32
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [4]:
# Load data
dataset_path = '/content/drive/MyDrive/PBL-3/Dataset'
train_generator = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        classes=['Acne and Rosacea Photos', 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions', 'Atopic Dermatitis Photos', 'Bullous Disease Photos', 'Eczema Photos', 'Melanoma Skin Cancer Nevi and Moles', 'Nail Fungus and other Nail Disease', 'Systemic Disease', 'Urticaria Hives', 'Vascular Tumors'])


Found 8796 images belonging to 10 classes.


In [5]:
# Create VGG19 model
vgg19_model = VGG19(include_top=False, input_shape=(224, 224, 3), pooling='max')
model = Sequential()
model.add(vgg19_model)
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(train_generator.num_classes, activation='softmax'))

# Freeze pre-trained layers
for layer in vgg19_model.layers:
    layer.trainable = False


80134624/80134624 [==============================] - 0s 0us/step


In [ ]:
# Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train model
model.fit_generator(train_generator,
                    steps_per_epoch=train_generator.samples // batch_size,
                    epochs=10)

<ipython-input-6-0f68a8e0fab9>:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/10
274/274 [==============================] - 1781s 6s/step - loss: 1.9935 - accuracy: 0.3231
Epoch 2/10
236/274 [========================>.....] - ETA: 23s - loss: 1.8532 - accuracy: 0.3689

In [ ]:
# Load and predict test image
test_image = 'trial.jpg'
img = image.load_img(test_image, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
preds = model.predict(x)
print(preds)